### 用于特征提取和图像匹配变换

In [1]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [23]:
def sobel(image):
    # 使用Sobel滤波器计算水平和垂直梯度
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    # 合并水平和垂直梯度
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
    return gradient_magnitude

# 1. 特征提取
def extract_features(image):
    # image_array = np.array(image)
    feature_extractor = cv2.SIFT_create()
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# 2. 特征匹配
def match_features_bf(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

def match_features_flann(descriptors1, descriptors2):
    # FLANN 参数设置
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    # 创建 FLANN 匹配器
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # 使用 KNN 进行匹配
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    # 选择好的匹配
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    return good_matches
# 注掉的方法是基于透视变换
# # 3. 变换估计
# def estimate_transform(keypoints1, keypoints2, good_matches):
#     src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     # 使用RANSAC算法估计变换矩阵
#     M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
#     return M

# #4. 变换应用
# def apply_transform(image, M):
#     # 应用估计的变换矩阵到图像
#     transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
#     return transformed_image

# 3. 仿射变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计仿射变换矩阵
    M = cv2.estimateAffinePartial2D(src_pts, dst_pts)[0]
    return M

# 4. 仿射变换应用
def apply_transform(image, M):
    # 应用估计的仿射变换矩阵到图像
    # image = Image.fromarray(image)
    transformed_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return transformed_image


In [42]:
# 一次配准
image1 = cv2.imread('origin_images/2.png', cv2.IMREAD_GRAYSCALE).astype(np.float64)
image2 = cv2.imread('origin_images/222.png', cv2.IMREAD_GRAYSCALE).astype(np.float64)

array = np.load('anti.npy')
array = Image.fromarray(array)
image1 = image1/array
image2 = image2/array
image1 = np.clip(image1*255,0,255).astype(np.uint8)
image2 = np.clip(image2*255,0,255).astype(np.uint8)
# image1 = Image.fromarray(img1)
# image2 = Image.fromarray(img2)

# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)

# 2. 特征匹配
good_matches = match_features_flann(descriptors1, descriptors2)

# 3. 变换估计
M = estimate_transform(keypoints1, keypoints2, good_matches)

# 4. 变换应用
transformed_image = apply_transform(image1, M)
cv2.imwrite('trans_image/transformed2to222.png', transformed_image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\sift.dispatch.cpp:512: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'cv::SIFT_Impl::detectAndCompute'


In [ ]:
# # 二次配准
# image11 = transformed_image
# # 1. 特征提取
# keypoints11, descriptors11 = extract_features(image11)
# # 2. 特征匹配
# good_matches2 =match_features(descriptors11, descriptors2)
# # 3. 变换估计
# M = estimate_transform(keypoints11, keypoints2, good_matches2)
# # 4. 变换应用
# transformed_image = apply_transform(image11, M)
# cv2.imwrite('2transformed2to222.png', transformed_image)

### 后面是SIFT和BFMatch合理性判断

In [ ]:
#一次配准
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)

img3 = cv2.drawKeypoints(image1,keypoints1,image1,color=(255,0,255))
img4 = cv2.drawKeypoints(image2,keypoints2,image2,color=(255,0,255))
hmerge = np.hstack((img3, img4))
# pillow_image = Image.fromarray(cv2.cvtColor(hmerge, cv2.COLOR_BGR2RGB))
# pillow_image.show()

In [ ]:
# BFMatcher解决匹配
#可能还得试试其他的匹配方法和
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1,descriptors2, k=2)
#print(matches)
#print(len(matches))
good = []
for m,n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])
img5 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,matches,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

pillow_image = Image.fromarray(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [ ]:
img6 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,good,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

pillow_image = Image.fromarray(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
pillow_image.show()